## 关系拟合

#### 建立数据集

In [ ]:
import torch
from torch.autograd import Variable
import matplotlib.pyplot as plt

# unsqueeze把以为数据变成二维，torch只能处理二维数据
x = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1)  # x data (tensor), shape=(100, 1)
# y是x二次方+噪点
y = x.pow(2) + 0.2*torch.rand(x.size())                 # noisy y data (tensor), shape=(100, 1)

# 用 Variable 来修饰这些数据 tensor
x, y = torch.autograd.Variable(x), Variable(y)

# 画图
plt.scatter(x.data.numpy(), y.data.numpy())
plt.show()

#### 创建神经网络的通用模式

In [ ]:
class Net(torch.nn.Module):  # 继承 torch 的 Module
    def __init__(self):
        super(Net, self).__init__()     # 继承 __init__ 功能
        
    def forward(self, x):   # 这同时也是 Module 中的 forward 功能
        pass

#### 建立神经网路 + 训练神经网络

In [ ]:

import torch
from torch.autograd import Variable
import torch.nn.functional as F     # 激励函数都在这
import matplotlib.pyplot as plt


#### 建立数据集
# unsqueeze把以为数据变成二维，torch只能处理二维数据
x = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1)  # x data (tensor), shape=(100, 1)
# y是x二次方+噪点
y = x.pow(2) + 0.2*torch.rand(x.size())                 # noisy y data (tensor), shape=(100, 1)

# 用 Variable 来修饰这些数据 tensor
x, y = torch.autograd.Variable(x), Variable(y)


#### 建立神经网络
class Net(torch.nn.Module):  # 继承 torch 的 Module
    ##### 定义
    def __init__(self, n_feature, n_hidden, n_output):# n_feature, n_hidden, n_output分别是输入层、隐藏层、输出层
        super(Net, self).__init__()     # 继承 __init__ 功能
        # 定义每层用什么样的形式
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # 隐藏层线性输出
        self.predict = torch.nn.Linear(n_hidden, n_output)   # 输出层线性输出
    #### 搭建
    def forward(self, x):   # 这同时也是 Module 中的 forward 功能
        # 正向传播输入值, 神经网络分析出输出值
        x = F.relu(self.hidden(x))      # 激励函数(隐藏层的线性值)
        # 预测时这里不用激励函数
        x = self.predict(x)             # 输出值
        return x

net = Net(n_feature=1, n_hidden=10, n_output=1)# 输入值：1；隐藏层：10；输出层：1

print(net)  # net 的结构


plt.ion()   # 设置plt为实时打印的方式
plt.show()


#### 训练网络
# optimizer 是训练的工具
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)  # 优化器： 传入 net 的所有参数, lr是学习率
loss_func = torch.nn.MSELoss()      # 预测值和真实值的误差计算公式 (均方差)

for t in range(100):
    prediction = net(x)     # 喂给 net 训练数据 x, 输出预测值

    loss = loss_func(prediction, y)     # 计算两者的误差；prediction在前，真实值y在后

    optimizer.zero_grad()   # 清空上一步的残余更新参数值；（将optimizer的梯度设为0）
    loss.backward()         # 误差反向传播, 计算参数更新值（反向传递）
    optimizer.step()        # 将参数更新值施加到 net 的 parameters 上（以学习效率0.5优化步骤）
    
    # 每5步plt显示一次，显示神经网络的训练过程
    if t % 5 == 0:
        plt.cla()
        plt.scatter(x.data.numpy(), y.data.numpy())# 原始数据
        plt.plot(x.data.numpy(), prediction.data.numpy(), 'r-', lw=5)# 预测数据
        # 当前误差
        plt.text(0.5, 0, 'Loss=%.4f' % loss.data[0], fontdict={'size': 20, 'color':  'red'})
        plt.pause(0.1)  


# plt.ioff()
# plt.show()
    
    


#### 训练网络